In [3]:
!pip install -q opencv-python matplotlib


In [3]:
import os, cv2, csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:

from cv2 import GaussianBlur

# Generates a RGB histogram for a single image
def plot_rgb_histogram(image_path):    
    ddepth = cv2.CV_16S
    kernel_size = 3
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) # Load an image

    # Apply Gaussian Blur - reduce noise
    image = cv2.GaussianBlur(image, (3, 3), 0)
    
    # Apply Laplace function
    dst = cv2.Laplacian(image, ddepth, ksize=kernel_size) 
    
    # converting back to uint8
    abs_dst = cv2.convertScaleAbs(dst)
    
    hist_gray = cv2.calcHist([abs_dst], [0], None, [256], [0, 256])
    return hist_gray

def process_images_from_folder(folder_path):
    if not os.path.exists(folder_path):
        print("Error: Specified folder does not exists.")
        return
    
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg', 'webp', 'tiff'))]
    if not image_files:
        print("No images found in the folder.")
        return
    
    histograms_list = []
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        hist = plot_rgb_histogram(image_path)  
        histograms_list.append((image_file, hist))
    
    distance_matrix = pd.DataFrame(0, index=image_files, columns=image_files, dtype='float64')

    # Calculating distance between images histograms
    for i in range(len(histograms_list)):
        for j in range(i, len(histograms_list)):
            image1, hist1 = histograms_list[i]
            image2, hist2 = histograms_list[j]
            
            total_distance = 0
            #total_distance +=  cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL) 
            total_distance +=  cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA) #distance between histograms
            #total_distance = 1 - total_distance # distance = 0 means images are equals

            distance_matrix.loc[image1, image2] = total_distance #distance values
            distance_matrix.loc[image2, image1] = total_distance
            
    distance_matrix.to_csv(os.path.join(folder_path,"distance_matrix.csv"))

#folder_path = "datasetSuddiviso/UV50"
folder_path = "datasetSuddiviso"

subfolders = [f for f in os.listdir(folder_path)]
print(subfolders)
for subfolder in subfolders:
    if subfolder == ".DS_Store":
        continue  # Salta il file .DS_Store
    process_images_from_folder(os.path.join(folder_path,subfolder))

Error: Specified folder does not exists.


'folder_path = "datasetSuddiviso"\n\nsubfolders = [f for f in os.listdir(folder_path)]\nprint(subfolders)\nfor subfolder in subfolders:\n    if subfolder == ".DS_Store":\n        continue  # Salta il file .DS_Store\n    process_images_from_folder(os.path.join(folder_path,subfolder))'